## Author: Mayank Yadav

---

### Overview

This module provides functions to fetch data from various APIs across multiple domains.

### Supported API Domains

1. **Police and Crime**
    - Crime head-wise IPC Cases during 2016, 18, 19, 20, 21
2. **Agriculture**
    - Crops
    - Fertilizers
    - Irrigation
    - Planning
    - Soil & Water Conservation  
3. **Water**
4. **Art and Culture**
5. **Defense**
6. **Education**
7. **Economy**
8. **Administration**
9. **Health and Family Welfare**
10. **Industry**
    - Tourism
11. **Infrastructure**
    - Roads
    - Railways
12. **Labour and Employment**
13. **Rural Development**


In [1]:

import requests
import json
import pandas as pd
import os


api_key = "579b464db66ec23bdd000001948d55cf357f47cc74b80568eae5c51a"

In [11]:
for index, f in enumerate(os.listdir("crime_raw_csv_data")):
    with open(f"crime_raw_csv_data/{f}", 'r') as csv_file:
        temp_data = pd.read_csv(csv_file);
        print("="*50)
        print(f"{index+1}. {f.replace(".csv", "")}:")
        for col_index, col_name in enumerate(list(temp_data.columns)):
            print(f"\t{col_index+1}. {col_name}")
        print("="*50)

1. Type of Explosive-wise Seizures_Recoveries under Explosives & Explosive Substances Act during 2021:
	1. Sl. No.
	2. Type of Explosive
	3. Seized from Extremists/ Insurgents/ Terrorists)
	4. Seized from Others (From Others Criminals/ Smugglers etc.)
	5. Total
2. Crime Head-wise Age Group and Gender-wise Persons Arrested under IPC Crimes in Metropolitan Cities during 2021:
	1. Sl. No.
	2. Crime Head
	3. Juveniles Apprehended - Boys
	4. Juveniles Apprehended - Girls
	5. Juveniles Apprehended - Transgender
	6. Juveniles Apprehended - Total
	7. 18 Years & Above - Below 30 Years - Male 
	8. 18 Years & Above - Below 30 Years - Female
	9. 18 Years & Above - Below 30 Years - Transgender
	10. 18 Years & Above - Below 30 Years - Total
	11. 30 Years & above - Below 45 Years - Male
	12. 30 Years & above - Below 45 Years - Female
	13. 30 Years & above - Below 45 Years - Transgender
	14. 30 Years & above - Below 45 Years - Total
	15. 45 Years & above - Below 60 Years - Male 
	16. 45 Years & above 

7. Crime Head-wise Court Disposal of IPC Crime Cases during 2021:
	1. Sl. No. Col. (1)
	2. Crime Head Col. (2)
	3. Cases Pending Trial from the Previous Year - Col. (3)
	4. Cases Sent for Trial during the year - Col. (4)
	5. Total Cases for Trial (Col.3+Col.4) - Col. (5)
	6. Cases Abated by Court - Col. (6)
	7. Cases Withdrawn from Prosecution - Col. (7)
	8. Cases Compounded or Compromised - Col. (8)
	9. Cases Disposed off by Plea Bargaining - Col. (9)
	10. Cases Quashed - Col. (10)
	11. Cases Disposed off without Trial (Col.6+Col.7+Col.8+Col.9+Col.10) - Col. (11)
	12. Cases Stayed or Sent to Record Room - Col. (12)
	13. Cases Convicted Out of Cases from Previous Year - Col. (13)
	14. Cases Convicted Out of Cases during the Year - Col. (14)
	15. Cases Convicted (Col.13+Col.14) - Col. (15)
	16. Cases Discharged - Col. (16)
	17. Cases Acquitted - Col. (17)
	18. Cases which Trials were Completed (Col.15+Col.16+Col.17) - Col. (18)
	19. Cases Disposed off by Courts (Col.11+Col.18) - Col. (1

In [134]:
crime_data_apis = {
    "State/UT-wise Number of Indian Penal Code (IPC) Crimes from 2020 to 2022": "/resource/15150682-a9ed-475d-b0e3-67b292e90d22",
    
# Metropolitan Cities
    
    # age and gender
    "City, Age Group and Gender-wise Persons Arrested under IPC Crimes in Metropolitan Cities during 2019": "/resource/f28fb4fd-86cb-49e5-a222-6776bf371fdd",
    "Age Group and Gender-wise Persons Arrested under IPC Crimes (in Metropolitan Cities) during 2020": "/resource/28c1b00d-3d75-46e8-ab03-f1c7bbaff11c",
    "Age Group and Gender-wise Persons Arrested under IPC Crimes (in Metropolitan Cities) during 2021": "/resource/6377dda7-c8b3-4e9b-8d88-ff1a281ce2b0",
    "Age Group and Gender-wise Persons Arrested under SLL Crimes (in Metropolitan Cities) during 2021": "/resource/219730ea-c97d-4e98-a62a-17876774c85b",
    
    # against women
    "City & Crime Head-wise IPC and SLL Crimes against Women in Metropolitan Cities during 2017": "/resource/5e7e25eb-4cc6-475d-893f-40fc50884eff",
    "Crime Head-wise Disposal of Persons Arrested under Indian Penal Code (IPC) Crimes in Metropolitan Cities during 2021": "/resource/4f17823c-dd32-460a-9b33-a4b2b4402bf4",
    
    "Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2017": "/resource/c8b12c25-e68f-4a27-a10f-3fb3cf05f4e3",
    "Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2021": "/resource/e45d1d52-cfb0-462f-8da6-23bda260077c",
    "Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2020": "/resource/b8cd95ac-9bec-46a1-a94d-2fab4a66cb2a",
    "Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2019": "/resource/9ba5ccdb-25de-4e03-8985-49cb06bb1662",
    "Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2018":"/resource/b7311014-31af-47da-b006-7fa4a38ca64b",
    
    "Crime Head-wise Court Disposal of Crime against Women in Metropolitan Cities during 2017": "/resource/e64f41ef-92f9-437a-9cc4-253db907aacf",
    "Crime Head-wise Court Disposal of Crime against Women in Metropolitan Cities during 2021": "/resource/a21dcf9c-c4e4-4973-bf9f-7a7dadccd7c0",
    "Crime Head-wise Court Disposal of Crime against Women in Metropolitan Cities during 2020": "/resource/ec646c30-45a4-4189-9b82-2ec27e5549fa",
    "Crime Head-wise Court Disposal of Crime against Women in Metropolitan Cities during 2019": "/resource/e25348f1-2186-4880-a133-b7488a7b1d1c",
    "Crime Head-wise Court Disposal of Crime against Women in Metropolitan Cities during 2018": "/resource/e7bc51a4-26c2-4c05-8f39-e338cdcfc66f",





    # crime head, age, and gender
    "Crime Head-wise Age Group and Gender-wise Persons Arrested under IPC Crimes in Metropolitan Cities during 2021": "/resource/6dccff77-d7ee-4dab-95d8-c4961b2eed48",
    "Crime Head-wise Age Group and Gender-wise Persons Arrested under SLL Crimes in Metropolitan Cities during 2021": "/resource/b2c4fbfe-4d22-4496-ad35-8b2238f29083",
    
    # crime head wise  IPC and SLL
    "State/City-wise Disposal of Persons Arrested under Indian Penal Code and Special & Local Laws (IPC & SLL) Crimes (in Metropolitan Cities) during 2021": "/resource/1a941c5e-f1f4-4c3e-8615-74850875f614",
    

# Court Disposals
    "Crime Head-wise Court Disposal of IPC Crime Cases during 2021": "/resource/2f29bd18-59d8-4f3e-9888-c6a0501a738e",
    "Crime Head-wise Court Disposal of SLL Crime Cases during 2021": "/resource/592e4555-9ba1-421c-aa36-9ae3a4636757",
    "State/UT-wise Court Disposal of IPC Crime Cases during 2021": "/resource/6f13237f-f727-4af9-97f1-6a417209b19e",
    "State/UT-wise Court Disposal of SLL Crime Cases during 2021": "/resource/b1f559c0-6a1b-4214-88b5-aaff2188f088",

    "Crime Head-wise Court Disposal of Crime against Women during 2017": "/resource/7c851b31-a0b3-4e49-9e88-c8bdd9d17449",
    "Crime Head-wise Court Disposal of Crime against Women during - 2021": "/resource/7f49af73-bbeb-484d-a1cc-f91981deff19",
    "Crime Head-wise Court Disposal of Crime against Women during 2020": "/resource/ddbc0642-6caf-4b3e-b141-98e1274bd14c",
    "Crime Head-wise Court Disposal of Crime against Women during 2019": "/resource/dc1fe545-076b-4d7d-9aec-0b26337ee54b",
    "Crime Head-wise Court Disposal of Crime against Women during 2018": "/resource/3c9d19e4-da1a-4df4-8062-f5f67e7881e3",



# Against Foreigners 
    "Crime Head-wise Crime against Foreigners during 2016": "/resource/c40f19dc-2f38-4bed-b093-2506622c984a",
    "Crime Head-wise Crime against Foreigners during 2018": "/resource/9056fe93-ca71-4100-b0a9-02e244313607",
    "Crime Head-wise Crime against Foreigners during 2019": "/resource/b6a43056-ea8e-48dc-b5f8-10ca22ed3c9b",
    "Crime Head-wise Crime against Foreigners during 2020": "/resource/3b30418e-f6d9-4c2a-a1d0-57762820eb41",
    "Crime Head-wise Crime against Foreigners during 2021": "/resource/f174cf36-1a1a-4b87-8c22-33741791326b",
    
# Police Disposals
    "Crime Head-wise Police Disposal of IPC Crime Cases during 2016": "/resource/4759cf71-d6dd-4b37-80b1-e0a10e45d90b",
    "Crime Head-wise Police Disposal of IPC Crime Cases during 2018": "/resource/08e68544-76b1-4596-a09e-47047abb26e2",
    "Crime Head-wise Police Disposal of IPC Crime Cases (Crime Head-wise) during 2019": "/resource/9fed9e21-d687-4c24-aa6e-63cb24bb68d3",
    "Crime Head-wise Police Disposal of IPC Crime Cases during 2020": "/resource/c73b59c9-04ed-4f03-98ed-4001d006962f",
    "Crime Head-wise Police Disposal of IPC Crime Cases during 2021": "/resource/8562bdac-58f1-40f3-9f32-b5f657e872a5",
    "Crime Head-wise Police Disposal of SLL Crime Cases during 2021": "/resource/7af701d0-53e5-42f9-aa52-8419bd30d59f",
    "Police Disposal of SLL Crimes in Metropolitan Cities during 2021": "/resource/fabf043a-e3e0-49a4-8c1e-372eba6e0370",
    "State/UT-wise Police Disposal of SLL Crime Cases during 2021": "/resource/305e23b5-67f2-4c31-8fd8-b05b49bb453b",

    "Crime Head-wise Police Disposal of Crime against Children during 2017": "/resource/e6deda82-8920-4954-841e-c4042729e971",
    "Crime Head-wise Police Disposal of Crime against Children during 2018": "/resource/bca3bd56-cd96-4df1-aec1-71ac46a9aabf",
    "Crime Head-wise Police Disposal of Crime against Children during 2019": "/resource/e0502065-0ba5-4ceb-9089-8993f9702fda",
    "Crime Head-wise Police Disposal of Crime against Children during 2020":"/resource/a2732898-e9cf-46fa-81be-8b713eb9dbfb",
    "Crime Head-wise Police Disposal of Crime against Children during 2021": "/resource/a5207328-d1ce-446c-9bbd-e65e9715a609",

    "Crime Head-wise Police Disposal of Crime against Women during 2021": "/resource/f7fd4d97-6bb0-40cd-90fb-0ed55a93c211",
    "Crime Head-wise Police Disposal of Crime against Women during 2020": "/resource/a502be00-c723-4f33-9cdf-af98d00fda21",
    "Crime Head-wise Police Disposal of Crime against Women during 2019": "/resource/c8968c89-0549-429f-a456-5ae01e51fa18",
    "Crime Head-wise Police Disposal of Crime against Women during 2018": "/resource/b7e71ce1-a8d7-48bd-8cda-c298d2a8edc3",
    "Crime Head-wise Police Disposal of Crime against Women during 2017": "/resource/8aab0d11-03ea-43bb-91f4-794d23f176d5",


# Against women
    "State/City-wise Indian Penal Code (IPC) Crimes against Women (Crime Head & City) during 2021": "/resource/eb37b5f8-671f-4a52-b528-c20d39797f43",
    "Crime against Women during 2014": "/resource/de6ddfff-c5b9-4058-a54a-776cee56567f",
    "Crime against Women during 2001-2012": "/resource/0171c62a-5304-43fa-83ff-8c9f16033e41",
    "Crime against Women during 2013": "/resource/ed286df7-e6bf-4938-8892-188cdeb1e62e",


# Against Children
    "Crime head-wise persons arrested under crime against children during 2013": "/resource/11626229-2afb-4950-8000-69d309c4269c",

# Arrests related
    "Persons Arrested under Various Sections of Cr.PC during 2021": "/resource/999d1451-4fae-48f9-9aec-586d87e9b3d6",  
    "Crime Head-wise Preventive Detentions during 2021": "/resource/75f413ea-9ccc-42ea-a2ff-f9ea7541f2b6",
    "State/UT-wise Caste of Convicts as on 31st December, 2020": "/resource/9cc08576-e504-44e8-9a63-251df05ce993",
    "State/UT-wise Religion of Convicts as on 31st December, 2020": "/resource/9241a606-688b-42b2-baea-c32916ad65cc",
    "State/UT-wise Domicile of Convicts as on 31st December, 2020": "/resource/1c64a0a2-4d9c-4bb9-8824-3b88dd9a3782",
    "State/UT-wise Education Profile of Convicts as on 31st December, 2020": "/resource/d0e86fbd-0e3f-44e8-9ac6-c412d5b66221",
    "State/UT-wise and Sex-wise Types of Prison Inmates in Special Jails as on 31st December, 2020": "/resource/7b5d0fc3-2b6c-4ade-8de8-268328f19546",
    "State/UT-wise Caste of Detenues as on 31st December, 2020": "/resource/2f742f78-6c6b-4b33-a6dd-7dfbe2efc795",
    "State/UT-wise Education Profile of other Prisoners as on 31st December, 2020": "/resource/c5d15711-07c2-4001-bb24-14b8b0ac645f",



    
# Kills
    "State/UT-wise All Uniform Police Personnel Injured on Duty during 2021": "/resource/52b4a360-991e-4683-992a-bc39b5279d21",
    "State/UT-wise All Uniform Police Personnel Killed on Duty during 2021": "/resource/c7002ab6-d7a6-4f5c-a26d-a7dbdacf92d4",
    "State/UT-wise Civilians Killed during Police Operations during 2021": "/resource/e1ce0691-9c6c-4e3e-bc62-586afb06653b",
    "All Uniform Police Personnel Killed/Injured on Duty during 2021": "/resource/7a816151-f9ce-4c19-9155-91fc6a52e6e3",

# Seizures and Recoveries
    "State/UT-wise Seizures under Arms Act during 2021": "/resource/34bf38a5-f348-4c06-a022-9408ae50eb3c",
    "State/UT-wise Seizure of Fake Indian Currency during 2021": "/resource/79c0d20c-272c-4fbd-a344-24d8304c6442",
    "Nature of Property-wise Value of Property Stolen & Recovered (in Crores) and Percentage Recovery during 2021": "/resource/56d8f4e1-098d-46ed-b1e8-4e9b054d08c6",
    "Drug - Head-wise Seizures under Narcotic Drugs and Psychotropic Substances NDPS Act during 2021": "/resource/7c4969d4-1601-4b77-8014-2d5cc0195aa5",
    "Type of Explosive-wise Seizures/Recoveries under Explosives & Explosive Substances Act during 2021": "/resource/d609a403-ee76-4f63-9abb-4ea10e99c7f3",

# Against Police
    "State/UT-wise Cases Registered against State Police Personnel during 2021": "/resource/edd8a94f-19c5-46b1-9985-2fdff15fd5aa",
    "Cases Registered against State Police Personnel for Human Right Violation during 2021": "/resource/f3ea8c03-90fd-4460-b116-447831ea2ad3",
}



In [135]:
def old_get_crime_data():
    """
    This uses the crime_data_apis to get the CSV and JSON data.
    It saves each response as <api_name>.json and <api_name>.csv.
    Returns a dictionary of raw CSV responses for each API.
    """
    params = {
        "api-key": f"{api_key}",
        "limit": "999",
        }
    
    extracted_csv_data = {}
    
    json_data_dir = "crime_raw_json_data"
    csv_data_dir = "crime_raw_csv_data"
    os.makedirs(csv_data_dir, exist_ok=True)
    os.makedirs(json_data_dir, exist_ok=True)
    
    for api_name, api_url in crime_data_apis.items():
        api_url = f"https://api.data.gov.in/{api_url}"
        print(f"\nDownloading data for {api_name}...")
        try:
            # Get JSON data
            json_response = requests.get(api_url,
                                         params={**params, "format": "json"},
                                        #  verify=False # avoid SSL
                                         timeout=20)
            json_response.raise_for_status()
            json_data = json_response.json()
            
            # Save JSON file
            json_path = os.path.join(json_data_dir, f"{api_name.replace('\\', '_')}.json")
            with open(json_path, "w") as f:
                json.dump(json_data, f, indent=2)
            print(f"Saved JSON for {api_name.replace('\\', '_')} to {json_path}")
            
            # Get CSV data
            csv_response = requests.get(api_url,
                                        params={**params, "format": "csv"},
                                        #  verify=False # avoid SSL
                                        timeout=20)
            csv_response.raise_for_status()
            csv_data = csv_response.text
            
            # Save CSV file
            csv_path = os.path.join(csv_data_dir, f"{api_name.replace('\\', '_')}.csv")
            with open(csv_path, "w") as f:
                f.write(csv_data)
            print(f"Saved CSV for {api_name.replace('\\', '_')} to {csv_path}")
            
            # Return CSV raw response in memory
            extracted_csv_data[api_name] = csv_data
        
        except Exception as e:
            print(f"Error while fetching data for {api_name}: {e}")
    
    return extracted_csv_data

In [136]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import itertools
import sys
import time
import os
import json
import requests
import re
from typing import Dict, Any

def sanitize_filename(name: str, max_len: int = 200) -> str:
    """
    Replace characters that are invalid in filenames across platforms.
    Also trims the name to max_len characters.
    """
    # replace forward/back slashes and other forbidden chars with underscore
    safe = re.sub(r'[\\/:"*?<>|]+', '_', name)
    # collapse whitespace and strip
    safe = re.sub(r'\s+', ' ', safe).strip()
    # optionally limit length
    if len(safe) > max_len:
        safe = safe[:max_len]
    return safe

def _spinner(stop_event: threading.Event, prefix: str = "Working"):
    """Simple spinner runs until stop_event is set."""
    spinner = itertools.cycle(["|", "/", "-", "\\"])
    while not stop_event.is_set():
        sys.stdout.write(f"\r{prefix}... {next(spinner)}")
        sys.stdout.flush()
        time.sleep(0.12)
    # clear spinner line on exit
    sys.stdout.write("\r" + " " * 80 + "\r")
    sys.stdout.flush()

def _print_progress(completed: int, total: int):
    """Print a simple textual progress bar on one line."""
    bar_len = 30
    filled = int(bar_len * completed / total) if total else bar_len
    bar = "#" * filled + "-" * (bar_len - filled)
    sys.stdout.write(f"\rProgress: [{bar}] {completed}/{total}")
    sys.stdout.flush()
    if completed == total:
        sys.stdout.write("\n")

def _fetch_one_api(session: requests.Session, api_name: str, api_url: str,
                   params: Dict[str, Any], json_dir: str, csv_dir: str,
                   verify_ssl: bool) -> Dict[str, Any]:
    """
    Worker that fetches JSON and CSV for one API and writes files.
    Returns a dict with {'api_name': api_name, 'csv': csv_text} on success.
    Raises exception on failure so caller can handle/log it.
    """
    # build full url if needed
    if not api_url.startswith("http"):
        api_url = f"https://api.data.gov.in/{api_url.lstrip('/')}"
    sanitized = sanitize_filename(api_name)
    # JSON
    jr = session.get(api_url, params={**params, "format": "json"}, timeout=30, verify=verify_ssl)
    jr.raise_for_status()
    json_data = jr.json()

    json_path = os.path.join(json_dir, f"{sanitized}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)

    # CSV
    cr = session.get(api_url, params={**params, "format": "csv"}, timeout=30, verify=verify_ssl)
    cr.raise_for_status()
    csv_text = cr.text

    csv_path = os.path.join(csv_dir, f"{sanitized}.csv")
    # ensure newline handling is consistent
    with open(csv_path, "w", encoding="utf-8", newline="") as f:
        f.write(csv_text)

    return {"api_name": api_name, "csv": csv_text, "json_path": json_path, "csv_path": csv_path}

def get_crime_data(crime_data_apis: Dict[str, str],
                   api_key: str,
                   max_workers: int = 8,
                   limit: int = 999,
                   json_data_dir: str = "crime_raw_json_data",
                   csv_data_dir: str = "crime_raw_csv_data",
                   verify_ssl: bool = True) -> Dict[str, str]:
    """
    Concurrently fetch JSON+CSV for each API in crime_data_apis.

    Returns:
      extracted_csv_data: dict mapping api_name -> raw csv text
    """
    os.makedirs(json_data_dir, exist_ok=True)
    os.makedirs(csv_data_dir, exist_ok=True)

    params = {"api-key": api_key, "limit": str(limit)}
    extracted_csv_data: Dict[str, str] = {}

    total_tasks = len(crime_data_apis)  # number of APIs (each API does both json&csv internally)
    completed = 0
    lock = threading.Lock()

    stop_event = threading.Event()
    spinner_thread = threading.Thread(target=_spinner, args=(stop_event, "Fetching APIs"), daemon=True)
    spinner_thread.start()

    # Use session for TCP connection reuse
    with requests.Session() as session:
        # optional headers (accept json by default)
        session.headers.update({"Accept": "application/json, text/csv;q=0.9, */*;q=0.1"})

        with ThreadPoolExecutor(max_workers=max_workers) as exe:
            future_to_name = {
                exe.submit(_fetch_one_api, session, name, url, params, json_data_dir, csv_data_dir, verify_ssl): name
                for name, url in crime_data_apis.items()
            }

            try:
                for fut in as_completed(future_to_name):
                    api_name = future_to_name[fut]
                    try:
                        res = fut.result()
                        extracted_csv_data[api_name] = res["csv"]
                        msg = f"Done: {api_name} (saved: {res['json_path']}, {res['csv_path']})"
                    except Exception as exc:
                        msg = f"Error for {api_name}: {exc}"
                    # update progress
                    with lock:
                        completed += 1
                        # print a short status line (this will not break spinner because spinner clears line)
                        _print_progress(completed, total_tasks)
                        # show last status message on the next line
                        sys.stdout.write(f"{msg}\n")
                        sys.stdout.flush()
            finally:
                stop_event.set()
                spinner_thread.join()

    return extracted_csv_data

In [137]:
def crime_head_wise_IPC_data_2016(data):
    pass
    

In [138]:
def crime_head_wise_IPC_data_2018(data):
    pass

In [139]:
def crime_head_wise_IPC_data_2019(data):
    pass

In [140]:
def crime_head_wise_IPC_data_2020(data):
    pass

In [141]:
def crime_head_wise_IPC_data_2021(data):
    pass

In [ ]:
try:
    # crime_extracted_data = get_crime_data()
    crime_extracted_data = get_crime_data(crime_data_apis, api_key, max_workers=8, verify_ssl=True)
except Exception as e:
    print(f"Some error occured in getting the data from data.gov.in")
    print(f"The error is {e}")
    

Fetching APIs... \

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.data.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.data.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.data.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverif

Fetching APIs... /

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.data.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.data.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.data.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverif

Progress: [------------------------------] 1/77Done: Age Group and Gender-wise Persons Arrested under IPC Crimes (in Metropolitan Cities) during 2020 (saved: crime_raw_json_data/Age Group and Gender-wise Persons Arrested under IPC Crimes (in Metropolitan Cities) during 2020.json, crime_raw_csv_data/Age Group and Gender-wise Persons Arrested under IPC Crimes (in Metropolitan Cities) during 2020.csv)
Progress: [------------------------------] 2/77Done: State/UT-wise Number of Indian Penal Code (IPC) Crimes from 2020 to 2022 (saved: crime_raw_json_data/State_UT-wise Number of Indian Penal Code (IPC) Crimes from 2020 to 2022.json, crime_raw_csv_data/State_UT-wise Number of Indian Penal Code (IPC) Crimes from 2020 to 2022.csv)
Progress: [#-----------------------------] 3/77Done: Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2017 (saved: crime_raw_json_data/Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2017.json, c

...

Progress: [###########################---] 71/77Done: State/UT-wise Seizure of Fake Indian Currency during 2021 (saved: crime_raw_json_data/State_UT-wise Seizure of Fake Indian Currency during 2021.json, crime_raw_csv_data/State_UT-wise Seizure of Fake Indian Currency during 2021.csv)
Progress: [############################--] 72/77Done: Drug - Head-wise Seizures under Narcotic Drugs and Psychotropic Substances NDPS Act during 2021 (saved: crime_raw_json_data/Drug - Head-wise Seizures under Narcotic Drugs and Psychotropic Substances NDPS Act during 2021.json, crime_raw_csv_data/Drug - Head-wise Seizures under Narcotic Drugs and Psychotropic Substances NDPS Act during 2021.csv)
Progress: [############################--] 73/77Done: Nature of Property-wise Value of Property Stolen & Recovered (in Crores) and Percentage Recovery during 2021 (saved: crime_raw_json_data/Nature of Property-wise Value of Property Stolen & Recovered (in Crores) and Percentage Recovery during 2021.json, crime_ra

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.data.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.data.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Progress: [############################--] 74/77Done: Type of Explosive-wise Seizures/Recoveries under Explosives & Explosive Substances Act during 2021 (saved: crime_raw_json_data/Type of Explosive-wise Seizures_Recoveries under Explosives & Explosive Substances Act during 2021.json, crime_raw_csv_data/Type of Explosive-wise Seizures_Recoveries under Explosives & Explosive Substances Act during 2021.csv)
Progress: [#############################-] 75/77Done: State/UT-wise Cases Registered against State Police Personnel during 2021 (saved: crime_raw_json_data/State_UT-wise Cases Registered against State Police Personnel during 2021.json, crime_raw_csv_data/State_UT-wise Cases Registered against State Police Personnel during 2021.csv)
Progress: [#############################-] 76/77Done: Cases Registered against State Police Personnel for Human Right Violation during 2021 (saved: crime_raw_json_data/Cases Registered against State Police Personnel for Human Right Violation during 2021.jso

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.data.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Progress: [##############################] 77/77
Done: State/UT-wise Seizures under Arms Act during 2021 (saved: crime_raw_json_data/State_UT-wise Seizures under Arms Act during 2021.json, crime_raw_csv_data/State_UT-wise Seizures under Arms Act during 2021.csv)


In [144]:
_ = 0
for api_name, data in crime_extracted_data.items():
    _ += 1
    print(f"{_}. {api_name}")

1. Age Group and Gender-wise Persons Arrested under IPC Crimes (in Metropolitan Cities) during 2020
2. State/UT-wise Number of Indian Penal Code (IPC) Crimes from 2020 to 2022
3. Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2017
4. Age Group and Gender-wise Persons Arrested under IPC Crimes (in Metropolitan Cities) during 2021
5. Crime Head-wise Disposal of Persons Arrested under Indian Penal Code (IPC) Crimes in Metropolitan Cities during 2021
6. City & Crime Head-wise IPC and SLL Crimes against Women in Metropolitan Cities during 2017
7. City, Age Group and Gender-wise Persons Arrested under IPC Crimes in Metropolitan Cities during 2019
8. Age Group and Gender-wise Persons Arrested under SLL Crimes (in Metropolitan Cities) during 2021
9. Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2019
10. Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2021
11. Crime Head-wise Police

In [ ]:
os.listdir